In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

#pd.options.display.float_format = '{:.1f}'.format  # 1 decimal only for float numbers display
#np.set_printoptions(suppress=True)

holidays_events = pd.read_csv("./database/holidays_events.csv", parse_dates=['date'])
test = pd.read_csv("./database/test.csv",parse_dates=['date'])
transactions = pd.read_csv("./database/transactions.csv",parse_dates=['date'])
stores = pd.read_csv("./database/stores.csv")
oil = pd.read_csv("./database/oil.csv",parse_dates=['date'])

transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [46]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [47]:
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
date           350 non-null datetime64[ns]
type           350 non-null object
locale         350 non-null object
locale_name    350 non-null object
description    350 non-null object
transferred    350 non-null bool
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB


In [48]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
date            83488 non-null datetime64[ns]
store_nbr       83488 non-null int64
transactions    83488 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [49]:
oil.rename(columns={'dcoilwtico':'oilprice'},inplace=True)

oil.head()


,date,oilprice
0,2013-01-01,nan
1,2013-01-02,93.1
2,2013-01-03,93.0
3,2013-01-04,93.1
4,2013-01-07,93.2


In [50]:
oil=oil.dropna(subset=['oilprice'],inplace=True)


In [53]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [54]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
